Вы планируете оценить следующую модель:

$$
lbwght_i = \beta_0 + \beta_1 \cdot male_i + \beta_2 \cdot parity_i + \beta_3 \cdot lfaminc_i + \beta_4 \cdot cigs_i + \varepsilon_i
$$


In [1]:
import pandas as pd
import numpy as np
import pyreadstat
from scipy import stats
from statsmodels.stats.diagnostic import het_white, het_breuschpagan
import statsmodels.api as sm
from linearmodels.iv import IV2SLS, compare
from linearmodels.shared.hypotheses import WaldTestStatistic as wald_test
# убрать предупреждения
import warnings
warnings.filterwarnings("ignore")

## Задание 1

### Пункт с)

Оцените с помощью OLS модель из пункта (a). Проинтерпретируйте полученные результаты.

In [5]:
df, _ = pyreadstat.read_dta('bwght.dta')
df

,faminc,cigtax,cigprice,bwght,fatheduc,motheduc,parity,male,white,cigs,lbwght,bwghtlbs,packs,lfaminc
0,13.5,16.5,122.300003,109,12,12,1,1,1,0,4.691348,6.8125,0.0,2.602690
1,7.5,16.5,122.300003,133,6,12,2,1,0,0,4.890349,8.3125,0.0,2.014903
2,0.5,16.5,122.300003,129,NaN,12,2,0,0,0,4.859812,8.0625,0.0,-0.693147
3,15.5,16.5,122.300003,126,12,12,2,1,0,0,4.836282,7.8750,0.0,2.740840
4,27.5,16.5,122.300003,134,14,12,2,1,1,0,4.897840,8.3750,0.0,3.314186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1383,27.5,30.0,138.300003,110,12,12,4,1,1,0,4.700480,6.8750,0.0,3.314186
1384,5.5,30.0,138.300003,146,NaN,16,2,1,1,0,4.983607,9.1250,0.0,1.704748
1385,65.0,8.0,118.599998,135,18,16,2,0,1,0,4.905275,8.4375,0.0,4.174387
1386,27.5,8.0,118.599998,118,NaN,14,2,0,1,0,4.770685,7.3750,0.0,3.314186


In [6]:
factors = sm.add_constant(df[['male', 'parity', 'lfaminc', 'cigs']])
lbwght = df['lbwght']
res = sm.OLS(lbwght, factors).fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                 lbwght   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     12.55
Date:                Sun, 16 Mar 2025   Prob (F-statistic):           4.90e-10
Time:                        22:46:06   Log-Likelihood:                 356.03
No. Observations:                1388   AIC:                            -702.1
Df Residuals:                    1383   BIC:                            -675.9
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.6756      0.022    213.681      0.0

In [54]:
print(f'lbwght = {res.params['const']:0.3f} + {res.params['male']:0.3f} * male + {res.params['parity']:0.3f} * parity + {res.params['lfaminc']:0.3f} * lfaminc + {res.params['cigs']:0.3f} * cigs')

lbwght = 4.676 + 0.026 * male + 0.015 * parity + 0.018 * lfaminc + -0.004 * cigs


### Пункт d)

Используя в качестве инструментальной переменной среднюю стоимость сигарет ($cigprice$), оцените модель из пункта (a) с помощью 2SLS. Сравните полученный результат с результатом из пункта (c).

In [7]:
iv_2sls = IV2SLS(
    dependent = lbwght,
    exog = factors.drop(['cigs'], axis=1),
    endog = factors['cigs'],
    instruments = df[['cigprice']]
).fit()
print(iv_2sls.summary)

                          IV-2SLS Estimation Summary                          
Dep. Variable:                 lbwght   R-squared:                     -1.8118
Estimator:                    IV-2SLS   Adj. R-squared:                -1.8199
No. Observations:                1388   F-statistic:                    10.018
Date:                Sun, Mar 16 2025   P-value (F-stat)                0.0401
Time:                        22:46:11   Distribution:                  chi2(4)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          4.4679     0.2559     17.463     0.0000      3.9664      4.9693
male           0.0298     0.0172     1.7348     0.08

### Пункт e)

Каким свойствам должна удовлетворять инструментальная переменная? На уровне значимости 5\% проверьте их для инструментальной переменной из пункта (d), описав подробно используемые тесты.

In [8]:
#Хаусман 
iv_2sls.wu_hausman(['cigs'])

Wu-Hausman test of exogeneity
H0: Variables cigs are exogenous
Statistic: 1.9186
P-value: 0.1662
Distributed: F(1,1382)
WaldTestStatistic, id: 0x3210de4b0

In [9]:
#F-статистика первого шага
dependent = df['cigs']
exog = sm.add_constant(df[['male', 'parity', 'lfaminc', 'cigprice']])
mod = sm.OLS(dependent, exog)
res = mod.fit()
print(res.summary())

                            OLS Regression Results                            
Dep. Variable:                   cigs   R-squared:                       0.030
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     10.86
Date:                Sun, 16 Mar 2025   Prob (F-statistic):           1.14e-08
Time:                        22:46:48   Log-Likelihood:                -4428.2
No. Observations:                1388   AIC:                             8866.
Df Residuals:                    1383   BIC:                             8892.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.7482      2.080      1.321      0.1

In [10]:
hypotheses = '(cigprice = 0)'
f_test = res.f_test(hypotheses)
print(f_test)

<F test: F=1.0018004472161448, p=0.31705033061915966, df_denom=1.38e+03, df_num=1>
